In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split

In [2]:
from Model import Feedforward
from dataset import FeatureDataset

30


In [3]:
#GOABAL VARIABLES
EPOCH = 10
LEARNING_RATE = 0.001
MOMENTUM = 0.2

In [4]:
#dataset load
DATASET_FOLDER = 'datasetv2/'
DATASET_FILE = DATASET_FOLDER+'dataset_for_mlp_full.csv'
init_dataset = FeatureDataset(DATASET_FILE)
lengths = [6240,1561]
train_dataset, test_dataset = random_split(init_dataset,lengths)
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=16, shuffle=True)

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [6]:
#load model
model = Feedforward(30,15,1)
model.to(device)

print(model)

Feedforward(
  (fc1): Linear(in_features=30, out_features=15, bias=True)
  (fc2): Linear(in_features=15, out_features=1, bias=True)
  (fc3): Linear(in_features=1, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
  (sigmoid): Sigmoid()
  (batchnorm1): BatchNorm1d(15, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [7]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [8]:
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [9]:
model.train()

Feedforward(
  (fc1): Linear(in_features=30, out_features=15, bias=True)
  (fc2): Linear(in_features=15, out_features=1, bias=True)
  (fc3): Linear(in_features=1, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
  (sigmoid): Sigmoid()
  (batchnorm1): BatchNorm1d(15, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [10]:
for e in range(1, EPOCH+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        
        y_pred = model(X_batch)
        
        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f}')

Epoch 001: | Loss: 0.43094 | Acc: 93.844
Epoch 002: | Loss: 0.25778 | Acc: 97.033
Epoch 003: | Loss: 0.15332 | Acc: 98.331
Epoch 004: | Loss: 0.10227 | Acc: 98.551
Epoch 005: | Loss: 0.08118 | Acc: 98.685


In [16]:
torch.save(model.state_dict(),'VPNClassifier')

# Testing

In [11]:
from sklearn.metrics import confusion_matrix, classification_report

In [12]:
test_loader = DataLoader(dataset=test_dataset,batch_size=1)

In [13]:
y_pred_list = []
y_test = []
model.eval()
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        y_test_pred = model(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_pred_list.append(y_pred_tag.cpu().numpy())
        y_test.append(y_batch.cpu().numpy())

y_pred_list = [a.squeeze().tolist() for a in y_pred_list]
y_test = [a.squeeze().tolist() for a in y_test]

In [14]:
confusion_matrix(y_test,y_pred_list)

array([[1476,    0],
       [   3,   82]])

In [15]:
print(classification_report(y_test,y_pred_list))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      1476
         1.0       1.00      0.96      0.98        85

    accuracy                           1.00      1561
   macro avg       1.00      0.98      0.99      1561
weighted avg       1.00      1.00      1.00      1561

